### Increasing the Maize Leaf Disease Dataset Using Contrast Technique

In [1]:
import PIL
import numpy as np
import os
import tensorflow as tf
import cv2
import keras
import pathlib
import glob
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Steps
- Loading the images and converting them into pixels.
- Resizing and Rescaling.
> Resizing involves making the images to be of same size and shape. Rescalling involves making the pixels to be in the range of 0-1.
- Coming up with a pre-processing Layer.
- Applying the pre-processing layer on the maize leaf disease dataset.
- Training the model.

### Task 1: Loading the images and converting them into pixels.

In [2]:

data_dir=pathlib.Path(os.getcwd()+"/maize_leaf_diseases_dataset/data/") # Windows Path
data_list=list(glob.glob(f"{data_dir}/*/*.jpg",recursive=True)) # Getting all the images path
print(len(data_list)) # Checking the length of our data

with open("maize_images_pixels.txt","w") as writer: # Creating a text file
    for imgPath in data_list: # Iterating through the list of image
        img_pxls=cv2.imread(str(imgPath)) # Image to pixels
        img=cv2.resize(img_pxls,(180,180))
#             print(img_pxls.shape)
#             print(img.shape)

        imgName=os.path.basename(imgPath) # Getting image name
        writer.write(f"{imgName}\n\n{img_pxls}\n\n") #Writin the image pixels in a text file


4186


### Labelling of the Maize Leaf Dataset

In [3]:
maize_labels_dict={
    "Blight":0,
    "Common Rust":1,
    "Gray Leaf Spot":2,
    "Healthy":3,
}
maize_leaf_dict={
    "Blight":list(glob.glob(f"{data_dir}/Blight/*.jpg")),
    "Common Rust":list(glob.glob(f"{data_dir}/Common_Rust/*.jpg")),
    "Gray Leaf Spot":list(glob.glob(f"{data_dir}/Gray_Leaf_Spot/*.jpg")),
    "Healthy":list(glob.glob(f"{data_dir}/Healthy/*.jpg")),
}


### Loading the images and turning image to pixels

In [4]:
x,y=[],[]
for maize_name,images in maize_leaf_dict.items():
    for image in images:
        img=cv2.imread(str(image))#Convert image to pixels
        resized_img=cv2.resize(img,(180,180))#Images to be of same size
        x.append(resized_img)#Pixels
        y.append(maize_labels_dict[maize_name])#Label Random 0,1,2,3
#     print(x[0])
#     print(y[:5])
x=np.array(x) #Pixels stored in numpy array
y=np.array(y) #Labels stored in numpy array


### Training, Test Split

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test=train_test_split(x,y,random_state=0)
print(len(x_train))
print(len(x_test))
x_train_scaled=x_train/255
x_test_scaled=x_test/255
#Scale
print(x_train_scaled[0])


3139
1047


In [6]:
num_classes=4 #Blight(0),Common Rust(1),Gray Leaf Spot(2),Healthy(3)
#Building the model CNN
#Activation function helps in predictions
model=Sequential([
    # 
    layers.Conv2D(16, 3, padding='same', activation='relu'),#Relu is our standard activation layer | applied 16 filters of 3 X 3 Matrix
    layers.MaxPooling2D(), # Reduce the size of the output by selecting the maximum value to increase computational speed 
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(), #2D array
    layers.Flatten(), # Flatten our values, the dense layer accepts a 1D array
    #128 neourons--> try and error
    layers.Dense(128,activation="relu"), #Dense perform matrix-vector multiplication
    layers.Dense(num_classes) #if 0 neuron is activated it means its Blight, no activation(linear activation)
])

In [7]:
# Compile the model
model.compile(optimizer="adam",
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #exit value is 4
              #loss-->exit value
              metrics=["accuracy"]
             )


In [8]:

#Update the weights in the network based on the input 
model.fit(x_train_scaled,y_train,epochs=1)
#Helps to determine whether the data is over trained or not
# Evaluate
print("Evaluate")
model.evaluate(x_test_scaled,y_test)
# Predictions
prediction=model.predict(x_test_scaled)
print(prediction)
#Convert to probability score
score=tf.nn.softmax(prediction[0])
print(score)
print(np.argmax(score))
print(y_test[0])
data_augmentation=Sequential([
    layers.experimental.preprocessing
])
plt.axis('off')
plt.imshow(x[0])


99/99 [==============================] - 55s 543ms/step - loss: 0.5938 - accuracy: 0.7646
Evaluate
33/33 [==============================] - 6s 158ms/step - loss: 0.5470 - accuracy: 0.7861
[[-1.7987967e-02 -1.5862657e+00 -5.8196443e-01  2.0177455e+00]
 [ 7.5370936e+00  1.8968250e+01  3.1816006e+00 -1.7378855e+01]
 [ 4.9515910e+00  2.6652369e+00  3.3730206e+00 -3.8650494e+00]
 ...
 [ 8.0318898e-02 -1.3762819e+00 -6.2900716e-01  1.9197729e+00]
 [ 5.7251792e+00  3.9352148e+00  4.5431533e+00 -5.9637847e+00]
 [ 7.6732254e+00  1.7796471e+01  5.3091598e+00 -1.8433516e+01]]
tf.Tensor([0.10598596 0.02208788 0.06029988 0.8116264 ], shape=(4,), dtype=float32)
3
3


TypeError: The added layer must be an instance of class Layer. Found: <module 'keras.api._v2.keras.layers.experimental.preprocessing' from 'C:\\Users\\User\\anaconda3\\lib\\site-packages\\keras\\api\\_v2\\keras\\layers\\experimental\\preprocessing\\__init__.py'>